In [113]:
import os
import pandas as pd

import toml

from dotenv import load_dotenv
load_dotenv()


config = toml.load("config.toml")

In [1]:
import pytubefix
from pytubefix import Playlist, Channel, YouTube

from _youtube import YouTubeVideo


In [114]:
def folder_csvs_to_excel(folder_path = "out", filename = "merged"):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    with pd.ExcelWriter(f"{folder_path}/{filename}.xlsx") as writer:
        for file in all_files:
            df = pd.read_csv(os.path.join(folder_path, file))
            sheet_name = os.path.splitext(file)[0][:31]  # Excel sheet name max length is 31
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [4]:
v = YouTube("https://www.youtube.com/watch?v=MBRqu0YOH14")

In [10]:
v.captions["en"].generate_txt_captions()

"Human existence is scary and confusing. A few hundred thousand years ago, we became conscious and found ourselves in a strange place. It was filled with other beings. We could eat some; some could eat us. There was liquid stuff we could drink; things we could use to make more things. The daytime sky had a tiny yellow ball that warmed our skin. The night sky was filled with beautiful lights. This place was obviously made for us. Something was watching over us. We were home. This made everything much less scary and confusing. But the older we got, the more we learned about the world and ourselves. We learned that the twinkling lights are not shining beautifully for us, they just are. We learned that we're not at the center of what we now call the universe, and that it is much, much older than we thought. We learned that we're made of many little dead things, which make up bigger things that are not dead, for some reason, and that we're just another temporary stage in a history going bac

In [58]:
import os
import wordcloud
import pandas as pd
from pathlib import Path

class Transcript:
    def __init__(self, video_url: str, language: str = "en"):
        self.video_url = video_url
        self.language = language
        
        self.video = YouTube(video_url)
        
        self.title = self.video.title
        self.title_no_special_chars = ''.join(e for e in self.title 
                                              if e.isalnum() or e.isspace()).replace(" ", "_")
        print(f"Video Title: {self.title_no_special_chars}")
    
        self.get_transcript(language=self.language)
    @staticmethod
    def srt_to_csv(srt_text: str) -> pd.DataFrame:
        """
        Convert SRT text to a DataFrame.
        """
        lines = srt_text.strip().split('\n')
        data = []
        for i in range(0, len(lines), 4):
            if i + 3 < len(lines):
                index = lines[i].strip()
                time_range = lines[i + 1].strip()
                text = lines[i + 2].strip()
                data.append({"index": index, "time_range": time_range, "text": text})
        return pd.DataFrame(data)
    
    @staticmethod
    def transcript_info(text: str) -> dict:
        """
        Extracts information from the transcript text.
        Returns a dictionary with the number of words and characters.
        """
        words = text.split()
        num_sentences = text.count('.') + text.count('!') + text.count('?')
        num_words = len(words)
        num_chars = len(text)
        
        return {
            "num_sentences": num_sentences,
            "num_words": num_words,
            "num_chars": num_chars
        }
    
    def get_available_languages(self):
        languages_available = self.video.captions.keys()
        if "a.hy" in languages_available:
            print("Automatic armenian subtitles are available.")
        if "en" in languages_available:
            print("English subtitles are available.")
        return languages_available
        
    
    
    def get_transcript(self, language: str = "en"):
        if language not in self.video.captions:
            msg = f"Language '{language}' not available. Available languages: {self.get_available_languages()}"
            print(msg)
            raise Exception(msg)
        
        self.caption = self.video.captions[language]
        self.text = self.caption.generate_txt_captions()
        self.srt = self.caption.generate_srt_captions()
        print(f"Transcript for '{language}' language has been generated.")
        print(f"Transcript info: {self.transcript_info(self.text)}")
        
        Path.mkdir(Path("videos", self.title_no_special_chars), exist_ok=True)
        
        filename = Path("videos", self.title_no_special_chars, self.title_no_special_chars)
        with open(f"{filename}.txt", "w", encoding="utf-8") as f:
            f.write(self.text)
        
        with open(f"{filename}.srt", "w", encoding="utf-8") as f:
            f.write(self.srt)
        print(f"Transcript saved to {filename}.txt and {filename}.srt")
        
        # Save srt to CSV
        self.srt_df = self.srt_to_csv(self.srt)
        csv_filename = f"{filename}.csv"
        self.srt_df.to_csv(csv_filename, index=False, encoding="utf-8")
        print(f"SRT converted to CSV and saved to {csv_filename}")
        
        # Generate word cloud
        wordcloud_filename = f"{filename}_wordcloud.png"
        wordcloud_image = wordcloud.WordCloud(width=800, height=400, background_color='white')
        wordcloud_image.generate(self.text)
        wordcloud_image.to_file(wordcloud_filename)
        print(f"Word cloud generated and saved to {wordcloud_filename}")
        
        return self.text, self.srt, self.srt_df    
          

In [59]:
t = Transcript("https://www.youtube.com/watch?v=MBRqu0YOH14")

Video Title: Optimistic_Nihilism
Transcript for 'en' language has been generated.
Transcript info: {'num_sentences': 78, 'num_words': 1075, 'num_chars': 5742}
Transcript saved to videos\Optimistic_Nihilism\Optimistic_Nihilism.txt and videos\Optimistic_Nihilism\Optimistic_Nihilism.srt
SRT converted to CSV and saved to videos\Optimistic_Nihilism\Optimistic_Nihilism.csv
Word cloud generated and saved to videos\Optimistic_Nihilism\Optimistic_Nihilism_wordcloud.png


In [48]:
t.get_transcript()

Transcript for 'en' language has been generated.
Transcript info: {'num_sentences': 78, 'num_words': 1075, 'num_chars': 5742}
Transcript saved to videos\Optimistic_Nihilism\Optimistic_Nihilism.txt and videos\Optimistic_Nihilism\Optimistic_Nihilism.srt
SRT converted to CSV and saved to videos\Optimistic_Nihilism\Optimistic_Nihilism.csv
Word cloud generated and saved to videos\Optimistic_Nihilism\Optimistic_Nihilism_wordcloud.png


("Human existence is scary and confusing. A few hundred thousand years ago, we became conscious and found ourselves in a strange place. It was filled with other beings. We could eat some; some could eat us. There was liquid stuff we could drink; things we could use to make more things. The daytime sky had a tiny yellow ball that warmed our skin. The night sky was filled with beautiful lights. This place was obviously made for us. Something was watching over us. We were home. This made everything much less scary and confusing. But the older we got, the more we learned about the world and ourselves. We learned that the twinkling lights are not shining beautifully for us, they just are. We learned that we're not at the center of what we now call the universe, and that it is much, much older than we thought. We learned that we're made of many little dead things, which make up bigger things that are not dead, for some reason, and that we're just another temporary stage in a history going ba

In [115]:
import os
import asyncio
import logging
from googletrans import Translator
# Set up logging to both file and console
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(name)s: %(message)s')

# File handler
file_handler = logging.FileHandler("translate.log", encoding="utf-8")
file_handler.setFormatter(log_formatter)
file_handler.setLevel(logging.INFO)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
console_handler.setLevel(logging.INFO)

# Root logger setup
logging.basicConfig(level=logging.INFO, handlers=[file_handler, console_handler])
logger = logging.getLogger(__name__)

class Translate:   
    def __init__(self, tr: Transcript, 
                 use_claude: bool = False, client=None, model: str = "claude-3-5-sonnet-20241022"):
        self.tr = tr
        self.text = tr.text
        self.df = tr.srt_df
        
        self.use_claude = use_claude
        self.client = client
        self.model = model
        
        self.google_translated_text = None
        self.google_translated_df = None
        self.claude_translated_text = None
        self.claude_translated_df = None
        
        self.PATH = Path("videos", tr.title_no_special_chars, "translations")
        Path.mkdir(self.PATH, exist_ok=True)
        
        if self.use_claude:
            logger.info(f"Using Claude for translation. | Model: {self.model}")
            if client is None:
                logger.info("No client provided, initializing Anthropic client.")
                from anthropic import Anthropic
                self.client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
        
    async def google_translate(self, target_lang: str = "hy"):
        async with Translator() as translator:
            res = await translator.translate(self.text, dest=target_lang)
            self.google_translated_text = res.text
            
            with open(self.PATH / "google.txt", "w", encoding="utf-8") as f:
                f.write(self.google_translated_text)
            logger.info("Google translation completed and saved to file.")
            return self.google_translated_text

    async def google_translate_df(self, target_lang: str = "hy") -> pd.DataFrame:
        """
        Translate the 'text' column of a DataFrame to the target language.
        """
        async with Translator() as translator:
            translations = await asyncio.gather(
                *[translator.translate(text, dest=target_lang) for text in self.df['text']]
            )
            self.df['google_translate'] = [t.text for t in translations]
            self.google_translated_df = self.df
            self.df.to_csv(self.PATH / "google.csv", index=False, encoding="utf-8")
        logger.info("Google DataFrame translation completed and saved to CSV.")
        return self.df
        
    @staticmethod
    def claude_request(client, model, prompt, messages):
        return client.messages.create(
            model=model,
            max_tokens=1024,
            system=prompt,
            messages=messages
        )    
    
    def claude_translate(self, target_lang: str = "en", experiment_mode: bool = True):
        if experiment_mode:
            text_to_translate = self.tr.text[:100]
        else:
            text_to_translate = self.tr.text
        
        prompt = f"Translate the following text to {target_lang}"
            
        self.response = self.claude_request(
            self.client, 
            self.model, 
            prompt, 
            [{"role": "user", "content": text_to_translate}]
        )   
        self.claude_text = self.response.content[0].text
        
        with open(self.PATH / "claude.txt", "w", encoding="utf-8") as f:
            f.write(self.claude_text)
        
        self.inp_tokens = self.response.usage.input_tokens
        self.out_tokens = self.response.usage.output_tokens
        
        logger.info(f"Claude translation completed. Input tokens: {self.inp_tokens} | Output tokens: {self.out_tokens}")
        
        return self.claude_text
    
    def claude_translate_df(self, target_lang: str = "en", experiment_mode: bool = True):
        if experiment_mode:
            df_to_translate = self.tr.srt_df.head(10)
        else:
            df_to_translate = self.tr.srt_df
        
        prompt = f"Translate the following text to {target_lang}"
        
        df_to_translate['claude_translation'] = df_to_translate['text'].apply(
            lambda text: self.claude_request(
                self.client, 
                self.model, 
                prompt, 
                [{"role": "user", "content": text}]
            ).content[0].text
        )
        self.claude_translated_df = df_to_translate
        
        df_to_translate.to_csv(self.PATH / "claude.csv", index=False, encoding="utf-8")
        logger.info("Claude DataFrame translation completed and saved to CSV.")
        
        return df_to_translate  
    
    @staticmethod
    def shorten_srt_range(ts_range: str) -> str:
        """
        Convert an SRT time range like
        "00:00:00,520 --> 00:00:04,019"
        into
        "00:00 -> 00:04"
        """
        pattern = r"(\d+):(\d+):(\d+),\d+\s*-->\s*(\d+):(\d+):(\d+),\d+"
        m = re.match(pattern, ts_range)
        if not m:
            raise ValueError(f"Invalid SRT time range: {ts_range!r}")

        sh, sm, ss, eh, em, es = m.groups()

        # combine hours into total minutes, then format as MM:SS
        start_total_min = int(sh) * 60 + int(sm)
        end_total_min   = int(eh) * 60 + int(em)

        start_fmt = f"{start_total_min:02d}:{int(ss):02d}"
        end_fmt   = f"{end_total_min:02d}:{int(es):02d}"

        return f"{start_fmt} -> {end_fmt}"
    
    def gather_translations(self):
        files = os.listdir(self.PATH)
        if not files:
            logger.warning("No translation files found in the directory.")
            raise FileNotFoundError("No translation files found in the directory.")
        
        expected_files = {"google.txt", "claude.txt", "google.csv", "claude.csv"}
        missing_files = expected_files - set(files)
        if missing_files:
            logger.warning(f"Missing translation files: {', '.join(missing_files)}")
        else:
            logger.info("All expected translation files are present.")
        
        df_google = pd.read_csv(self.PATH / "google.csv", encoding="utf-8")
        df_claude = pd.read_csv(self.PATH / "claude.csv", encoding="utf-8")
        text_google = open(self.PATH / "google.txt", "r", encoding="utf-8").read()
        text_claude = open(self.PATH / "claude.txt", "r", encoding="utf-8").read()
        
        df_google['claude_translation'] = df_claude['claude_translation']
        df_all = df_google.copy()
        
        # improve time range column ""00:00:03,520 --> 00:03""
        df_all["time_range"] = df_all["time_range"].apply(
            self.shorten_srt_range
        )
        
        df_full = pd.DataFrame({
            "text": self.tr.text,
            "google_translation": [text_google],
            "claude_translation": [text_claude],
        })
        
        new_folder = Path(self.PATH) / "combined"
        Path.mkdir(new_folder, exist_ok=True)
        df_all.to_csv(new_folder / "both_translations.csv", index=False, encoding="utf-8")
        df_full.to_csv(new_folder / "full_translations.csv", index=False, encoding="utf-8")
        
        folder_csvs_to_excel(new_folder, filename="translations_combined")
        
        
        
        
        

In [116]:
translator = Translate(t, use_claude=True)

INFO:__main__:Using Claude for translation. | Model: claude-3-5-sonnet-20241022
INFO:__main__:No client provided, initializing Anthropic client.


In [118]:
import re
translator.gather_translations()

INFO:__main__:All expected translation files are present.


In [109]:
translator.claude_translate("Armenian")

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:__main__:Claude translation completed. Input tokens: 34 | Output tokens: 125


"Here's the Armenian translation:\n\nՄարդկային գոյությունը վախեցնող և շփոթեցնող է։ Մի քանի հարյուր հազար տարի առաջ մենք գիտակից դարձանք և հայտ...\n\nNote: The source text appears to be incomplete, cutting off mid-sentence. I've translated what was provided, but if you'd like to share the complete text, I'll be happy to translate the full passage."

In [110]:
translator.claude_translate_df("Armenian")

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
C:\Users\hayk_\AppData\Local\Temp\ipykernel_15904\76328443.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,index,time_range,text,google_translate,claude_translation
0,1,"00:00:00,520 --> 00:00:04,019",Human existence is scary and confusing.,Մարդու գոյությունը սարսափելի է եւ շփոթեցնող:,Մարդկային գոյությունը վախեցնող և շփոթեցնող է։
1,2,"00:00:04,019 --> 00:00:09,600","A few hundred thousand years ago, we became co...",Մի քանի հարյուր հազար տարի առաջ մենք գիտակից դ...,Մի քանի հարյուր հազար տարի առաջ մենք գիտակից դ...
2,3,"00:00:09,600 --> 00:00:14,820",It was filled with other beings. We could eat ...,Այն լցված էր այլ էակներով: Մենք կարող էինք ուտ...,Այն լի էր այլ էակներով։ Մենք կարող էինք ուտել ...
3,4,"00:00:14,820 --> 00:00:19,500",There was liquid stuff we could drink; things ...,"Հեղուկ նյութեր կար, որ կարող էինք խմել; բաներ,...",Translation into Armenian:\n\nԿային հեղուկ նյո...
4,5,"00:00:19,500 --> 00:00:22,900",The daytime sky had a tiny yellow ball that wa...,"The երեկային երկինքը փոքրիկ դեղին գնդակ ուներ,...","Ցերեկային երկինքը փոքրիկ դեղին գնդակ ուներ, որ..."
5,6,"00:00:22,900 --> 00:00:25,620",The night sky was filled with beautiful lights.,Գիշերային երկինքը լցվեց գեղեցիկ լույսերով:,Գիշերային երկինքը լի էր գեղեցիկ լույսերով։
6,7,"00:00:25,620 --> 00:00:28,100",This place was obviously made for us.,Այս վայրն ակնհայտորեն արվեց մեզ համար:,Այս տեղն ակնհայտորեն ստեղծված է մեզ համար։
7,8,"00:00:28,100 --> 00:00:29,940",Something was watching over us.,Ինչ-որ բան հետեւում էր մեզ:,Ինչ-որ բան հսկում էր մեզ։
8,9,"00:00:29,940 --> 00:00:35,020",We were home. This made everything much less s...,Մենք տուն էինք: Սա ամեն ինչ շատ ավելի քիչ վախե...,Մենք տանն էինք: Սա ամեն ինչ շատ ավելի քիչ վախե...
9,10,"00:00:35,020 --> 00:00:38,960","But the older we got, the more we learned abou...","Բայց ավելի հինացանք, այնքան ավելի շատ իմացանք ...","Բայց որքան մեծանում էինք, այնքան ավելի շատ էին..."


In [111]:
await translator.google_translate()

INFO:httpx:HTTP Request: GET https://translate.googleapis.com/translate_a/single?client=gtx&sl=auto&tl=hy&hl=hy&dt=at&dt=bd&dt=ex&dt=ld&dt=md&dt=qca&dt=rw&dt=rm&dt=ss&dt=t&ie=UTF-8&oe=UTF-8&otf=1&ssel=0&tsel=0&tk=xxxx&q=Human+existence+is+scary+and+confusing.+A+few+hundred+thousand+years+ago%2C+we+became+conscious+and+found+ourselves+in+a+strange+place.+It+was+filled+with+other+beings.+We+could+eat+some%3B+some+could+eat+us.+There+was+liquid+stuff+we+could+drink%3B+things+we+could+use+to+make+more+things.+The+daytime+sky+had+a+tiny+yellow+ball+that+warmed+our+skin.+The+night+sky+was+filled+with+beautiful+lights.+This+place+was+obviously+made+for+us.+Something+was+watching+over+us.+We+were+home.+This+made+everything+much+less+scary+and+confusing.+But+the+older+we+got%2C+the+more+we+learned+about+the+world+and+ourselves.+We+learned+that+the+twinkling+lights+are+not+shining+beautifully+for+us%2C+they+just+are.+We+learned+that+we%27re+not+at+the+center+of+what+we+now+call+the+universe%2C+a

'Մարդու գոյությունը սարսափելի է եւ շփոթեցնող: Մի քանի հարյուր հազար տարի առաջ մենք գիտակից դարձանք եւ հայտնվեցինք տարօրինակ վայրում: Այն լցված էր այլ էակներով: Մենք կարող էինք ուտել մի քանիսը. Ոմանք կարող էին մեզ ուտել: Հեղուկ նյութեր կար, որ կարող էինք խմել; բաներ, որոնք մենք կարող էինք օգտագործել ավելի շատ բաներ անելու համար: The երեկային երկինքը փոքրիկ դեղին գնդակ ուներ, որը տաքացրեց մեր մաշկը: Գիշերային երկինքը լցվեց գեղեցիկ լույսերով: Այս վայրն ակնհայտորեն արվեց մեզ համար: Ինչ-որ բան հետեւում էր մեզ: Մենք տուն էինք: Սա ամեն ինչ շատ ավելի քիչ վախեցավ եւ շփոթեցնող: Բայց ավելի հինացանք, այնքան ավելի շատ իմացանք աշխարհի եւ մեր մասին: Տեղեկացանք, որ շողշողացող լույսերը գեղեցիկ չեն փայլում մեզ համար, նրանք պարզապես են: Մենք իմացանք, որ մենք այն կենտրոնում չենք, թե ինչն ենք այժմ անվանում տիեզերք, եւ որ այն շատ ավելի հին է, քան կարծում ենք: Մենք իմացանք, որ մենք պատրաստված ենք շատ փոքր մեռած բաներից, որոնք կազմում են ավելի մեծ բաներ, որոնք ինչ-ինչ պատճառներով մեռած չեն, եւ որ մենք եւս մեկ

In [ ]:
await translator.google_translate_df("hy")

AttributeError: 'Anthropic' object has no attribute 'chat'

In [ ]:
gt = await google_translate(t.text, "hy")

In [52]:
df_translated = await translate_df(t.srt_df, "hy")

In [53]:
df_translated

,index,time_range,text,translated_text
0,1,"00:00:00,520 --> 00:00:04,019",Human existence is scary and confusing.,Մարդու գոյությունը սարսափելի է եւ շփոթեցնող:
1,2,"00:00:04,019 --> 00:00:09,600","A few hundred thousand years ago, we became co...",Մի քանի հարյուր հազար տարի առաջ մենք գիտակից դ...
2,3,"00:00:09,600 --> 00:00:14,820",It was filled with other beings. We could eat ...,Այն լցված էր այլ էակներով: Մենք կարող էինք ուտ...
3,4,"00:00:14,820 --> 00:00:19,500",There was liquid stuff we could drink; things ...,"Հեղուկ նյութեր կար, որ կարող էինք խմել; բաներ,..."
4,5,"00:00:19,500 --> 00:00:22,900",The daytime sky had a tiny yellow ball that wa...,"The երեկային երկինքը փոքրիկ դեղին գնդակ ուներ,..."
...,...,...,...,...
86,87,"00:05:32,940 --> 00:05:37,040","If this is our one shot at life, there is no r...","Եթե ​​սա մեր մեկ կրակոցն է կյանքում, ապա պատճա..."
87,88,"00:05:37,040 --> 00:05:38,840",and live as happy as possible.,եւ ապրեք հնարավորինս երջանիկ:
88,89,"00:05:39,460 --> 00:05:42,400",Bonus points if you made the life of other peo...,"Բոնուսային միավորներ, եթե ավելի լավ եք դարձրել..."
89,90,"00:05:42,400 --> 00:05:45,980",More bonus points if you help build a galactic...,"Ավելի շատ բոնուսային միավորներ, եթե օգնում եք ..."


In [18]:
ma = Channel("https://www.youtube.com/@MetricAcademy")

In [19]:
# playlists = ma.playlists
# print(len(playlists))

In [ ]:
# for pl in playlists:
# 	print(pl.title)
    

In [21]:
ma = pytubefix.Channel("https://www.youtube.com/@MetricAcademy")

In [22]:
videos = ma.video_urls

In [ ]:
len(videos)

In [8]:
from _youtube import YouTubeVideo

video_url = "https://youtu.be/7XsIyFTImwk"

video = YouTubeVideo(video_url)
tr = video.get_transcript()
print(tr)
# save transcript to file
with open("transcript.srt", "w", encoding="utf-8") as f:
    f.write(tr)

Only the following languages are available: KeysView({})


ValueError: No captions available for language: a.hy

In [ ]:
# video.get_metadata()

In [ ]:
from openai import OpenAI

import os
import toml

from dotenv import load_dotenv
load_dotenv()


config = toml.load("config.toml")

client = OpenAI()


In [ ]:
from pydantic import BaseModel, ValidationError


In [ ]:
import re

class Segment(BaseModel):
    title: str
    start_time: str  # e.g. '00:03:15'

    @classmethod
    def __get_validators__(cls):
        yield from super().__get_validators__()
        yield cls.validate_time_format

    @staticmethod
    def validate_time_format(value):
        if not re.match(r"^\d{2}:\d{2}:\d{2}$", value):
            raise ValueError("start_time must be in HH:MM:SS format")
        return value
 
class TimeStamps(BaseModel):
    segments: list[Segment]

In [ ]:
prompt = """You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return \ntitle and start_time\n\n[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.\n\nYou need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary"""

In [ ]:
def generate_segmented_timestamps(transcript: str) -> str:
    """
    Uses the OpenAI chat endpoint (api-mode=chat) to analyze a transcript
    and return key segment titles with start timestamps as JSON.
    """
    # Create chat completion
    print(prompt)
    response = client.chat.completions.parse(
        model=config["openai"]["model"],
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": transcript
            }
        ],
        response_format=TimeStamps,
        
    )

    # Extract and return the assistant's content
    return response.choices[0].message.parsed


In [ ]:
with open("transcript.srt", "r") as f:
    transcript_text = f.read()

In [ ]:
res = generate_segmented_timestamps(transcript_text)

You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return 
title and start_time

[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.

You need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary


In [ ]:
for segment in res.segments:
    print(f"{segment.start_time} {segment.title}")


00:00:00 Ներածություն և պլան
00:14:00 Python գրադարաններ եւ գործիքներ
00:34:00 YouTube տվյալներ մշակելն ու ստանալը
01:00:00 Աուդիո և վիդեո ներբեռնում
01:14:00 Թարգմանական գործիքներ: Google Translate եւ Deepl
01:34:00 Նախապատրաստական եւ ամփոփում


# Antrophic

In [ ]:
from _youtube import YouTubeVideo

video_url = input("Enter YouTube video URL: ")

video = YouTubeVideo(video_url)
tr = video.get_transcript()

# save transcript to file
with open("transcript.srt", "w", encoding="utf-8") as f:
    f.write(tr)

import re

# ── configurable filters ────────────────────────────────────────────────
TIMESTAMP_RGX      = re.compile(r'^\d{2}:\d{2}:\d{2},\d{3}\s*-->', re.M)   # 00:00:00,000 -->
STAGE_DIR_RGX      = re.compile(r'^[\[(].*?[\])]$', re.M)                  # [Music], (laughs)
CUE_INDEX_RGX      = re.compile(r'^\d+\s*$', re.M)                         # 742
BLANK_LINE_RGX     = re.compile(r'^\s*$', re.M)                            # empty lines
# ────────────────────────────────────────────────────────────────────────

def clean_srt_text(srt_text: str, join_paragraphs: bool = False) -> str:
    """
    Strip cue numbers, timestamps, blank lines and bracketed
    stage directions from a YouTube­-generated SRT transcript.

    Parameters
    ----------
    srt_text : str
        Raw transcript exactly as read from an .srt file.
    join_paragraphs : bool, default=False
        If True, collapses all remaining lines into a single paragraph.

    Returns
    -------
    str
        Cleaned transcript text.
    """
    # Remove each class of “extra stuff”
    txt = TIMESTAMP_RGX.sub('', srt_text)
    txt = CUE_INDEX_RGX.sub('', txt)
    txt = STAGE_DIR_RGX.sub('', txt)
    txt = BLANK_LINE_RGX.sub('\n', txt)     # normalise multiple blank lines

    # Strip leading/trailing whitespace on each line
    lines = [ln.strip() for ln in txt.splitlines() if ln.strip()]
    lines = [lines[i] for i in range(1, len(lines), 4)]
    # take every 4th line (the actual text)
 

    return (' '.join(lines) if join_paragraphs else '\n'.join(lines))

# ── example usage ────────────────────────────────────────────────────────
if __name__ == "__main__":
    # imagine srt_text came from an HTTP request, clipboard, etc.
    with open("transcript.srt", encoding="utf-8") as f:
        srt_text = f.read()

    tr_clean = clean_srt_text(srt_text)              # multi-line output
    # print(clean_srt_text(srt_text, True))      # one long paragraph
tr_clean

'Ողջույն, էսօր տենց երեք կտորից ա դասը\nոր առաջին անգամ Python-ից դուրս կգանք,\nմինիմալոտ մակարդակի էն ամենա հիմնական\nկբռնի մի հատ ուրիշ բանի հետ, որ\nէն ա, որ\nենք\nբաներ են, որ ծրագրավորողը պետք է իմանադ\nֆայլերի հետ աշխատել ինչ-որ ֆայլեր բացել\nերրորդ բանն էլ այն որ կսովորենք մի քանի\nգրադարանների պրինցիպը աշխատելու որն ա ու\nներմուծել ու էդ տենց մի աշխարհ\nու մի հատ հետաքրքիր գրադարան կա որ\nօգտագործել, չենք ուսումնասիրել\nսահմանափակենք, ձեռով գրենք էն ինչ պետք\nայսօրվանից հետո արդեն մի քիչ այդ\nդուրս\nանցնենք կախեցզվեց\nկապված նախ ոնց բացենք ընդհանրապես\nտերմինալը տերմինալը կարանք սահմանենք\nաշխատացնելու գործիքս ասած ու օրինակի\nտեղ մի վայրկյան\nհատ թազա ֆայլ ֆայլի անունը փոխենք, մի\nայլն, և այլն, հեսա կտեսնենք ոնց կարանք\nինչը առաջին հերթին կարի բավականին\nերկրորդը կարի անհրաժեշտ ուղղակի, երբ որ\nորտեղ մենք ուղղակի էկրան չենք ունենալու,\nօրինակ, գանք, սպեն ինչ ուղղակի\nաշխատացվի։ Ըը ու հա, տերմինալ բացելու\nտերմինալ կամ cmd կամ կոնսոլ տենց տարբեր\nբան, որ եթե գաք ուղղակի ստեղ այ

: 

In [ ]:



prompt = """You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return \ntitle and start_time in the format [start_time] [title]\n\n[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.\n\nYou need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary. Return only a text with the segments in the format:
# 00:00 Segment Title ..."""


message = client.messages.create(
    # model="claude-opus-4-20250514",
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    system=prompt,
    messages=[
        {"role": "user", "content": tr_clean}
    ]
)

text = message.content[0].text


In [ ]:

print("Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել")
print(text.replace("#", ""))

Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել
Here are the key segments from the video lecture:

00:00:00 Գործնական խնդիր - Password գեներատոր
- Առաջադրանք՝ ստեղծել password գեներատոր ֆունկցիա 
- Տարբեր բարդության մակարդակներ (easy, medium, hard)
- Թվեր, տառեր և սիմվոլներ օգտագործելով

00:29:30 Random մոդուլի կիրառում
- Random մոդուլի հիմնական ֆունկցիաներ
- Random choice և uniform ֆունկցիաներ
- Պատահական թվերի գեներացում

00:47:00 Մեծ թվերի օրենքի օրինակ
- Կոպեկի գցման սիմուլյացիա 
- Հավանականությունների հաշվարկ
- Պատահականության վիզուալիզացիա գրաֆիկով

01:15:00 Matplotlib գրաֆիկների օգտագործում
- Pyplot մոդուլի import
- Գրաֆիկների կառուցում
- Տվյալների վիզուալիզացիա

01:30:00 Դասի ամփոփում և հաջորդ թեմաներ
- Գրադարանների օգտագործման նախապատրաստում
- Python-ի հնարավորությունների ընդլայնում
- Հետագա ուսուցման պլաններ


In [ ]:

# # extract ```json part
# json_part = text.split("```json")[1].split("```")[0].strip()
# import json
# res = json.loads(json_part)
# for segment in res:
# 	print(f"{segment['start_time']} {segment['title']}")

In [ ]:
# res = eval(message.content[0].text.replace("\n", " ").replace("  ", " ")[8:-4])

In [ ]:
# res

In [ ]:
print("Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել")
for segment in res:
    print(f"{segment['start_time']} {segment['title']}")


Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել
00:00:00 Գործնական դասերի ավարտ և հաջորդ թեմաների պլան
00:11:00 YouTube-ից վիդեո մետադատաների քաշում dataclass-ով
00:27:00 Transcript (ենթագրերի) քաշում և աշխատանք
00:33:00 Աուդիո և վիդեո ֆայլերի ներբեռնում YouTube-ից
00:52:00 Google Translate և DeepL API-ների հետ աշխատանք
01:13:00 Աբստրակտ կլասներ և թարգմանիչների Pipeline
01:26:00 Git և քոմիթների մասին քննարկում


In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.getenv("CLAUDE_API_KEY"),
)

# Playlist

In [6]:
import re
import pandas as pd
from tqdm import tqdm
from _youtube import YouTubeVideo
from pytubefix import Playlist, YouTube, Channel

In [7]:
ma = Channel("https://www.youtube.com/@MetricAcademy")

In [8]:
ma.playlists

[<pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxu2_l1zR1RpGQqT9fp-WIR>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUytC30DnVEKt-inCmsylOpY>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxfiN8AF5b-LFDvSrT_dVj6>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUzpXv7Bey_8qkVKPSMhN2cS>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwro_kavF8BvvJXmyXB7jVF>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwUa_o8Fxig8SlC1PrLYWUg>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxjP7WnHxAWfbd8LWSY-pix>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwbO3Ts-JzkLTRaFcpTBgNL>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwkKFfZYCeIip7BHzmk0mbK>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwNhdJWh3lpAJLFBjV6ilqv>]

In [9]:
len(ma.video_urls)

10

In [ ]:
for pl in tqdm(ma.playlists):
    pl_title = pl.title
    print(pl_title)
    pl_title_no_special_chars = re.sub(r'[^\w\s]', '', pl_title)
    print(pl_title_no_special_chars)
    
    res = []

    for v in tqdm(pl.video_urls):
        video = YouTubeVideo(v)
        
        data = video.get_metadata()
        res.append(data)
    
    res_df = pd.DataFrame(res)
    res_df.to_csv(f"out/{pl_title_no_special_chars}.csv", index=False)


  0%|          | 0/10 [00:00<?, ?it/s]

Python Libraries
Python Libraries


 10%|█         | 1/10 [00:06<00:55,  6.19s/it]

Tuple-ներ և Set-եր | 24 բաժին խնդիրներ | Python Profound
Tupleներ և Setեր  24 բաժին խնդիրներ  Python Profound


 20%|██        | 2/10 [00:28<02:07, 15.91s/it]

...

Չմոնտաժված տեսագրություններ
Չմոնտաժված տեսագրություններ


 90%|█████████ | 9/10 [07:39<00:50, 51.00s/it]

Python - Դասեր
Python  Դասեր


100%|██████████| 10/10 [09:40<00:00, 58.03s/it]


AttributeError: 'Channel' object has no attribute 'playlist_'

In [12]:

with open("out/export_info.txt", "w", encoding="utf-8") as f:
    f.write(f"Exported {len(ma.playlists)} playlists with {len(ma.video_urls)} videos.\n")
    f.write(f"Date: {pd.Timestamp.now()}\n")

In [13]:
# pandas merge all cvs on a folder into one excel file
import os
import pandas as pd

folder_path = "out"
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
with pd.ExcelWriter("out/merged_data.xlsx") as writer:
    for file in all_files:
        df = pd.read_csv(os.path.join(folder_path, file))
        sheet_name = os.path.splitext(file)[0][:31]  # Excel sheet name max length is 31
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Armenian transcript

In [4]:
vid = YouTubeVideo("https://youtu.be/7XsIyFTImwk")

tr = vid.get_transcript("a,hy", return_srt=False)
print(f"Num characters {len(tr)} | Num words {len(tr.split())} | Num sentences {len(tr.split('.'))}")


NameError: name 'YouTubeVideo' is not defined

In [14]:
from googletrans import Translator

async def google_translate(text, target_lang):
    async with Translator() as translator:
        result = await translator.translate(text, dest=target_lang)
        return result.text

In [15]:
translation = await google_translate(tr, "hy")

In [16]:
translation

'Այդ ցուրտ, անձրեւոտ օրերին, երբ մոռանում ես անձրեւի բաճկոնը կամ հովանոցը, եւ ուզում ես հնարավորինս չոր մնալ ... պետք է քայլես եւ ավելի շատ ժամանակ անցկացիր անձրեւի տակ: Կամ պետք է գործարկեք, ինչը նշանակում է, որ դուք կխորտակվեք ավելի շատ անձրեւաջրերի կողքից: Ենթադրելով, որ դուք դեռ լիովին ներծծված չեք եղել, եւ դուք չեք նետվում պուդլների մեջ, պատասխանը պարզ է: Երբ դուրս եք գալիս անձրեւաջրերի մեկ ճանապարհից, դուք անցնում եք մեկ ուրիշի ճանապարհով: Այսպիսով, ձեր գագաթին հարվածող անձրեւի քանակը կայուն է, անկախ այն բանից, թե որքան արագ եք գնում: Այլապես, կարող եք պատկերացնել, որ անձրեւներն իրենք են գրենական պիտույքներ, եւ դուք (եւ ձեր տակ գտնվող երկիրը) շարժվում են անձրեւի միջով: Եվ քանի որ զուգահեռացված (դա 3D զուգահեռ) ծավալը կախված չէ իր թեքությունից, այնուհետեւ անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, նույն քանակությամբ անձրեւի վրա կուղեւորվի ձեզանից յուրաքանչյուր վայրկյան: Հիմա, եթե դուք չեք շարժվում, վերեւից անձրեւը բոլորը կստանաք: Բայց եթե շարժվում ես, կողքից նույնպես վազ

In [38]:
# !pip install --upgrade google-api-python-client
!pip install google-genai

  Using cached google_genai-1.23.0-py3-none-any.whl.metadata (38 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
Using cached google_genai-1.23.0-py3-none-any.whl (223 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl (176 kB)

   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ------------- -------------------------- 1/3 [tenacity]
   -------------------------- ------------- 2/3 [google-genai]
   -------------------------- ------------- 2/3 [google-genai

In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
text = """Այն ցուրտ, անձրևոտ օրերին, երբ մոռանում եք ձեր անձրևանոցը և ցանկանում եք հնարավորինս չոր մնալ ... պետք է քայլե՞լ և ավելի շատ ժամանակ անցկացնել անձրևի տակ: Թե՞ պետք է վազել, ինչը նշանակում է, որ կբախվենք նաև կողքից ընկնող անձրևի կաթիլների հետ: 
...
Ենթադրելով, որ դուք դեռ ամբողջությամբ չեք թրջվել և չեք թռչում ջրակույտերի մեջ, պատասխանը պարզ է: Երբ դուք շարժվում եք մի ընկնող անձրևի կաթիլից դուրս, դուք շարժվում եք մեկ այլ կաթիլի ուղղությամբ: Այսպիսով, ձեզ վերևից հարվածող անձրևի կաթիլների քանակը հաստատուն է՝ անկախ նրանից, թե որքան արագ եք շարժվում: 
...
Որպես այլընտրանք՝ կարող եք պատկերացնել, որ անձրևօ կաթիլները անշարքժ են, իսկ դուք (ձեր տակ գտնվող երկրագնդի հետ միասին) շարժվում ենք վերև անձրևի միջով: 
...
Եվ քանի որ զուգահեռանիստի (նույն ինքը՝ երեք դ զուգահեռագծի) ծավալը կախված չէ նրա թեքությունից, ապա անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, ամեն վարկյան ձեր վրա ընկած անձևի քանակը նույնն է լինելու։
...
Եթե դուք չեք շարժվում, ապա վերևից ընկնող անձրևի կաթիլները միակն ենք որոնց կբախվենք: Բայց եթե շարժվում ենք, ապա նաև կբախվենք կողքից եկող անձրևի կաթլիներին և ավելի շատ կթրջվենք: Այսպիսով, ամեն վայրկյան, ամենաչորը մնալու համար պետք է անշարժ կանգնենք, և որքան ավելի արագ շարժվենք, այնքան ավելի շատ կթրջվենք:
...
Բայց եթե փորձում եք հասնել A կետից B կետ, ապա անշարժ կանգնելը լավագույն միտքը չէ: Եվ A կետից B կետ գնալու ժամանակ, անձրևի ընդհանուր քանակը, որին կբախվեք կողքից, կախված չէ նրանից, թե որքան արագ եք գնում - ճիշտ այնպես, ինչպես ձյունամաքրող մեքենան նույն քանակությամբ ձյուն կմաքրի ճանապարհի մի հատվածից՝ անկախ նրա արագությունից: Անձրևի միջով վազելու դեպքի համար կարող եք դա պարզել կրկին զուգահեռանիստների օգնությամբ:
...
Այսպիսով, ձեր ընդհանուր թրջվածությունը հավասար է ամեն վարկյան վերևից եկող անձրևի պատճառով թրջվածությանը՝ բազմապատկած անձրևի տակ անցկացրած ժամանակի քանակով, գումարած կողքից եկող անձրևի թրջվածությանը ամեն մետրի համար՝ բազմապատկած ձեր անցած մետրերի քանակով:
...
Հիշենք որ ֆիքսված ժամանակահատվածում նույն քանակությամբ անձրև կհարվածի ձեզ վերևից՝ անկախ նրանից, թե որքան արագ եք գնում: Եվ տվյալ հեռավորության վրա՝ նույն քանակությամբ անձրև կհարվածի կողքից - կրկին, անկախ նրանից, թե որքան արագ եք գնում:
...
Իմի բերելով, մի կետից մյուսը գնալիս ամենաչորը մնալու համար, պետք է փորձել նվազագույնի հասցնել վերևից ձեր վրա ընկնող ջրի քանակը: Եվ բավականին պարզ է, որ դա նշանակում է անձրևից հնարավորինս արագ փախչել:
"""

In [26]:
tr = text
print(f"Num characters {len(tr)} | Num words {len(tr.split())} | Num sentences {len(tr.split('.'))}")


Num characters 2326 | Num words 354 | Num sentences 4


In [27]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import mimetypes
import os
import re
import struct
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()
    print(f"File saved to to: {file_name}")


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro-preview-tts"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        response_modalities=[
            "audio",
        ],
        speech_config=types.SpeechConfig(
            voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                    voice_name="Zephyr"
                )
            )
        ),
    )

    file_index = 0
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if (
            chunk.candidates is None
            or chunk.candidates[0].content is None
            or chunk.candidates[0].content.parts is None
        ):
            continue
        if chunk.candidates[0].content.parts[0].inline_data and chunk.candidates[0].content.parts[0].inline_data.data:
            file_name = f"first_attmempt_{file_index}"
            file_index += 1
            inline_data = chunk.candidates[0].content.parts[0].inline_data
            data_buffer = inline_data.data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            if file_extension is None:
                file_extension = ".wav"
                data_buffer = convert_to_wav(inline_data.data, inline_data.mime_type)
            save_binary_file(f"{file_name}{file_extension}", data_buffer)
        else:
            print(chunk.text)

def convert_to_wav(audio_data: bytes, mime_type: str) -> bytes:
    """Generates a WAV file header for the given audio data and parameters.

    Args:
        audio_data: The raw audio data as a bytes object.
        mime_type: Mime type of the audio data.

    Returns:
        A bytes object representing the WAV file header.
    """
    parameters = parse_audio_mime_type(mime_type)
    bits_per_sample = parameters["bits_per_sample"]
    sample_rate = parameters["rate"]
    num_channels = 1
    data_size = len(audio_data)
    bytes_per_sample = bits_per_sample // 8
    block_align = num_channels * bytes_per_sample
    byte_rate = sample_rate * block_align
    chunk_size = 36 + data_size  # 36 bytes for header fields before data chunk size

    # http://soundfile.sapp.org/doc/WaveFormat/

    header = struct.pack(
        "<4sI4s4sIHHIIHH4sI",
        b"RIFF",          # ChunkID
        chunk_size,       # ChunkSize (total file size - 8 bytes)
        b"WAVE",          # Format
        b"fmt ",          # Subchunk1ID
        16,               # Subchunk1Size (16 for PCM)
        1,                # AudioFormat (1 for PCM)
        num_channels,     # NumChannels
        sample_rate,      # SampleRate
        byte_rate,        # ByteRate
        block_align,      # BlockAlign
        bits_per_sample,  # BitsPerSample
        b"data",          # Subchunk2ID
        data_size         # Subchunk2Size (size of audio data)
    )
    return header + audio_data

def parse_audio_mime_type(mime_type: str) -> dict[str, int | None]:
    """Parses bits per sample and rate from an audio MIME type string.

    Assumes bits per sample is encoded like "L16" and rate as "rate=xxxxx".

    Args:
        mime_type: The audio MIME type string (e.g., "audio/L16;rate=24000").

    Returns:
        A dictionary with "bits_per_sample" and "rate" keys. Values will be
        integers if found, otherwise None.
    """
    bits_per_sample = 16
    rate = 24000

    # Extract rate from parameters
    parts = mime_type.split(";")
    for param in parts: # Skip the main type part
        param = param.strip()
        if param.lower().startswith("rate="):
            try:
                rate_str = param.split("=", 1)[1]
                rate = int(rate_str)
            except (ValueError, IndexError):
                # Handle cases like "rate=" with no value or non-integer value
                pass # Keep rate as default
        elif param.startswith("audio/L"):
            try:
                bits_per_sample = int(param.split("L", 1)[1])
            except (ValueError, IndexError):
                pass # Keep bits_per_sample as default if conversion fails

    return {"bits_per_sample": bits_per_sample, "rate": rate}


if __name__ == "__main__":
    generate()


File saved to to: first_attmempt_0.wav


In [30]:
from pytubefix import YouTube

In [31]:
yt_orig = YouTube("https://www.youtube.com/watch?v=3MqYE2UuN24")

In [33]:
import requests

url = yt_orig.thumbnail_url
response = requests.get(url)
if response.status_code == 200:
    with open("thumbnail.jpg", "wb") as f:
        f.write(response.content)
    print("Thumbnail saved as thumbnail.jpg")
else:
    print("Failed to fetch thumbnail:", response.status_code)

Thumbnail saved as thumbnail.jpg


# Audio video download

In [5]:
from _youtube import YouTubeVideo


In [7]:
yt = YouTubeVideo("https://www.youtube.com/watch?v=3MqYE2UuN24")

yt.download_audio(output_path="running_in_the_rain")

In [8]:
yt.download_video(output_path="running_in_the_rain", highest_quality=True)

In [9]:
yt.download_transcript(output_path="running_in_the_rain", language="en")

In [17]:
transcript_text = yt.get_transcript("en", return_srt=False)


In [19]:
translation

'Այդ ցուրտ, անձրեւոտ օրերին, երբ մոռանում ես անձրեւի բաճկոնը կամ հովանոցը, եւ ուզում ես հնարավորինս չոր մնալ ... պետք է քայլես եւ ավելի շատ ժամանակ անցկացիր անձրեւի տակ: Կամ պետք է գործարկեք, ինչը նշանակում է, որ դուք կխորտակվեք ավելի շատ անձրեւաջրերի կողքից: Ենթադրելով, որ դուք դեռ լիովին ներծծված չեք եղել, եւ դուք չեք նետվում պուդլների մեջ, պատասխանը պարզ է: Երբ դուրս եք գալիս անձրեւաջրերի մեկ ճանապարհից, դուք անցնում եք մեկ ուրիշի ճանապարհով: Այսպիսով, ձեր գագաթին հարվածող անձրեւի քանակը կայուն է, անկախ այն բանից, թե որքան արագ եք գնում: Այլապես, կարող եք պատկերացնել, որ անձրեւներն իրենք են գրենական պիտույքներ, եւ դուք (եւ ձեր տակ գտնվող երկիրը) շարժվում են անձրեւի միջով: Եվ քանի որ զուգահեռացված (դա 3D զուգահեռ) ծավալը կախված չէ իր թեքությունից, այնուհետեւ անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, նույն քանակությամբ անձրեւի վրա կուղեւորվի ձեզանից յուրաքանչյուր վայրկյան: Հիմա, եթե դուք չեք շարժվում, վերեւից անձրեւը բոլորը կստանաք: Բայց եթե շարժվում ես, կողքից նույնպես վազ

In [22]:
import pandas as pd

pd.DataFrame({"english": [transcript_text],
              "translation": [translation]}).to_csv("running_in_the_rain/text.csv")

# Auto dubbing